# ACME: King County Home Renovation Investment Plan


## Overview

TBD

## Business Problem
TBD

1. Will renovations have a meaningful impact on home price?
2. If so, which 3 renovations will have the most outsized impact on price?
3. What will be the profit expected from these renovations? 

## Data Preperation

Importing the data

In [1]:
import pandas as pd

kc = pd.read_csv('data/kc_house_data.csv')

In [2]:
kc.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [3]:
kc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

Looks like we have 21,597 home records in our data. There is some missing data in waterfront, view, and yr_renovated.

In [8]:
# Checking how many unique values are in each column.

kc.nunique()

id               21420
date               372
price             3622
bedrooms            12
bathrooms           29
sqft_living       1034
sqft_lot          9776
floors               6
waterfront           2
view                 5
condition            5
grade               11
sqft_above         942
sqft_basement      304
yr_built           116
yr_renovated        70
zipcode             70
lat               5033
long               751
sqft_living15      777
sqft_lot15        8682
dtype: int64

The data came with a dictionary file which explains each column. I recreate that dictionary here for convenience, along with some additional datapoints.

### Data Dictionary

| **Column**        | Description                                                                      | Unique | Dtype   |
|:-----------------:|:--------------------------------------------------------------------------------:|:------:|:-------:|
| **id**            | Unique identifier for a house                                                    | 21420  | int64   |
| **date**          | Date house was sold                                                              | 372    | object  |
| **price**         | Sale price                                                                       | 3622   | float64 |
| **bedrooms**      | Number of bedrooms                                                               | 12     | int64   |
| **bathrooms**     | Number of bathrooms                                                              | 29     | float64 |
| **sqft_living**   | Square footage of living space in the home                                       | 1034   | int64   |
| **sqft_lot**      | Square footage of the lot                                                        | 9776   | int64   |
| **floors**        | Number of floors (levels) in house                                               | 6      | float64 |
| **waterfront**    | Whether the house is on a waterfront                                             | 2      | object  |
| **view**          | Quality of view from house                                                       | 5      | object  |
| **condition**     | How good the overall condition of the house is. Related to maintenance of house. | 5      | object  |
| **grade**         | Overall grade of the house. Related to the construction and design of the house. | 11     | object  |
| **sqft_above**    | Square footage of house apart from basement                                      | 942    | int64   |
| **sqft_basement** | Square footage of the basement                                                   | 304    | object  |
| **yr_built**      | Year when house was built                                                        | 116    | int64   |
| **yr_renovated**  | Year when house was renovated                                                    | 70     | float64 |
| **zipcode**       | ZIP Code used by the United States Postal Service                                | 70     | int64   |
| **lat**           | Latitude coordinate                                                              | 5033   | float64 |
| **long**          | Longitude coordinate                                                             | 751    | float64 |
| **sqft_living15** | The square footage of interior housing living space for the nearest 15 neighbors | 777    | int64   |
| **sqft_lot15**    | The square footage of the land lots of the nearest 15 neighbors                  | 8682   | int64   |

We have some location data thanks to the columns zipcode, lat and long.

However, these are not very intuative. To handle that, I looked up a list of Zip codes in King County and what City they correspond to. I import that data here, and then match it to our initial dataset.

This data came from [here](https://www.ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?FIPS=53033).

In [13]:
kc_zip_city = pd.read_csv('data/king-county-zip-code-city.csv')

In [14]:
kc_zip_city = dict(zip(kc_zip_city['Zip Code'], kc_zip_city.City))
kc['city'] = kc['zipcode'].apply(lambda x: kc_zip_city.get(x,'None'))
kc['city'].value_counts()

Seattle          6926
Renton           1597
Redmond          1418
Kent             1201
Shoreline        1194
Federal Way      1140
Issaquah         1092
Bellevue         1090
Kirkland          977
Tukwila           853
Maple Valley      589
Auburn            550
Bothell           478
Yarrow Point      317
Snoqualmie        308
Mercer Island     282
Woodinville       273
Enumclaw          233
North Bend        220
None              198
Duvall            190
Carnation         124
Vashon            117
Black Diamond     100
Fall City          80
Medina             50
Name: city, dtype: int64

Looks like out of 21,420 homes we couldn't match 198 to a city. It's a small enough number, so we'll drop those rows so it doesn't create issues later.

In [15]:
kc.drop(kc[kc['city'] == 'None'].index, inplace = True)

### Data Cleaning